In [15]:
import torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt

In [3]:
#Understand the structure of the dSprites dataset
data = np.load('datasets/dsprites_ndarray_co1sh3sc6or40x32y32_64x64.npz',
                encoding='latin1', # These two
                allow_pickle=True) # are for loading metadata
data

NpzFile 'datasets/dsprites_ndarray_co1sh3sc6or40x32y32_64x64.npz' with keys: metadata, imgs, latents_classes, latents_values

In [66]:
# imgs = torch.Tensor(data['imgs'])
len(imgs)

737280

In [14]:
k = torch.from_numpy(data['imgs'])
k = torch.reshape(k, (k.shape[0],-1))
print(k.shape, print(k.dtype))


out = torch.distributions.bernoulli.Bernoulli(probs = torch.clamp(k.float(), min = 1e-6, max = 1-1e-6))
print(torch.sum(out.entropy(),dim=0).shape)

torch.uint8
torch.Size([737280, 4096]) None
torch.Size([4096])


In [34]:
logit = torch.clamp(torch.randn(k.shape), min = 0.1, max = 0.9)
logit = torch.log(logit/(1-logit))
logit.shape

torch.Size([737280, 4096])

In [35]:
logit

tensor([[ 1.3481, -1.1151, -2.1972,  ...,  2.1972, -2.1972, -2.1972],
        [-2.1972,  0.6491,  2.1972,  ...,  0.9261, -2.1972,  2.1972],
        [-2.1972, -2.1972, -2.1972,  ..., -2.1972,  2.1972,  2.1972],
        ...,
        [-2.1972, -2.1972,  1.1770,  ..., -2.1972,  1.7386, -0.6148],
        [ 0.4552, -2.1972, -1.3803,  ...,  2.1972, -2.1972, -1.5469],
        [-2.1972, -0.9131, -2.1972,  ..., -2.1972,  0.4492,  2.1972]])

In [36]:
torch.mean(torch.sum(F.binary_cross_entropy_with_logits(logit, logit**3, reduction='none'),dim=1))

tensor(-69663.8438)

In [6]:
latent_sizes = torch.Tensor(data["metadata"][()]["latents_sizes"])
# torch.index_select(latent_sizes, 0,torch.tensor([0,1,2,3]))

In [7]:
latent_sizes

tensor([ 1.,  3.,  6., 40., 32., 32.])

In [35]:
factor_bases = torch.prod(latent_sizes.int())/torch.cumprod(latent_sizes.int(),axis=0)
factor_bases

tensor([7.3728e+05, 2.4576e+05, 4.0960e+04, 1.0240e+03, 3.2000e+01, 1.0000e+00])

In [44]:
samples = torch.rand(10, len(latent_sizes))
new_samples = (samples * (latent_sizes)).int().floor()

In [12]:
latent_sizes

tensor([ 1.,  3.,  6., 40., 32., 32.])

In [45]:
new_samples

tensor([[ 0,  0,  0, 21, 17, 22],
        [ 0,  0,  2, 13,  8, 22],
        [ 0,  2,  0,  5, 21,  6],
        [ 0,  1,  0,  8, 31, 16],
        [ 0,  0,  0, 26,  6, 18],
        [ 0,  0,  0,  9, 24,  3],
        [ 0,  0,  2, 17, 18,  9],
        [ 0,  2,  0, 30, 30,  6],
        [ 0,  2,  3, 27, 27, 17],
        [ 0,  1,  2, 24,  7,  9]], dtype=torch.int32)

In [22]:
indices = torch.tensor(np.dot(new_samples.numpy(), sample.numpy())).int()
indices

tensor([604108, 267977, 316720, 374397, 475746, 341568, 229797, 370521, 565354,
        457936], dtype=torch.int32)

In [63]:
print(new_samples)
print(factor_bases.int())
idxs = torch.matmul(new_samples, factor_bases.int())
print(idxs)
new_samples.shape, factor_bases.shape, torch.concatenate((imgs[idxs],imgs[idxs]),axis=1).unsqueeze(1).shape

tensor([[ 0,  0,  0, 21, 17, 22],
        [ 0,  0,  2, 13,  8, 22],
        [ 0,  2,  0,  5, 21,  6],
        [ 0,  1,  0,  8, 31, 16],
        [ 0,  0,  0, 26,  6, 18],
        [ 0,  0,  0,  9, 24,  3],
        [ 0,  0,  2, 17, 18,  9],
        [ 0,  2,  0, 30, 30,  6],
        [ 0,  2,  3, 27, 27, 17],
        [ 0,  1,  2, 24,  7,  9]], dtype=torch.int32)
tensor([737280, 245760,  40960,   1024,     32,      1], dtype=torch.int32)
tensor([ 22070,  95510, 497318, 254960,  26834,   9987,  99913, 523206, 642929,
        352489], dtype=torch.int32)


(torch.Size([10, 6]), torch.Size([6]), torch.Size([10, 1, 128, 64]))

In [58]:
torch.all(imgs[idxs][1] == imgs[95510])

tensor(True)

In [11]:
new_samples

tensor([[ 0,  1,  4,  7, 28, 11],
        [ 0,  2,  0, 29,  8, 21],
        [ 0,  2,  3,  5, 16, 19],
        [ 0,  1,  0,  5, 29, 19],
        [ 0,  0,  2, 38, 15, 17],
        [ 0,  0,  1, 29, 22, 21],
        [ 0,  2,  3, 19, 19, 26],
        [ 0,  1,  5, 17, 10, 11],
        [ 0,  0,  1, 35, 25, 15],
        [ 0,  1,  1, 36, 30, 17]])

In [59]:
new_samples[:,[1, 3 ,4]]

tensor([[ 1,  5,  2],
        [ 2, 21,  2],
        [ 1, 35, 16],
        [ 2,  5, 15],
        [ 0, 20, 26],
        [ 1, 26, 11],
        [ 2, 24, 23],
        [ 0, 37,  1],
        [ 1, 23, 26],
        [ 0, 35, 24]])

In [47]:
latent_sizes.shape

torch.Size([6])

In [35]:
# Get a list of random indices
indices = np.random.randint(0, 200, 5)
indices

array([138,  22,  84, 122, 104])

In [33]:
factor_sizes = np.array(data["metadata"][()]["latents_sizes"])
np.prod(factor_sizes)/ np.cumprod(factor_sizes) # number of images per factor

array([7.3728e+05, 2.4576e+05, 4.0960e+04, 1.0240e+03, 3.2000e+01,
       1.0000e+00])

In [32]:
imgs.shape

ValueError: can only specify one unknown dimension

In [25]:
data['metadata'][()]

{'date': 'April 2017',
 'description': 'Disentanglement test Sprites dataset.Procedurally generated 2D shapes, from 6 disentangled latent factors.This dataset uses 6 latents, controlling the color, shape, scale, rotation and position of a sprite. All possible variations of the latents are present. Ordering along dimension 1 is fixed and can be mapped back to the exact latent values that generated that image.We made sure that the pixel outputs are different. No noise added.',
 'version': 1,
 'latents_names': ('color', 'shape', 'scale', 'orientation', 'posX', 'posY'),
 'latents_possible_values': {'orientation': array([0.        , 0.16110732, 0.32221463, 0.48332195, 0.64442926,
         0.80553658, 0.96664389, 1.12775121, 1.28885852, 1.44996584,
         1.61107316, 1.77218047, 1.93328779, 2.0943951 , 2.25550242,
         2.41660973, 2.57771705, 2.73882436, 2.89993168, 3.061039  ,
         3.22214631, 3.38325363, 3.54436094, 3.70546826, 3.86657557,
         4.02768289, 4.1887902 , 4.34989

In [31]:
torch.randint(0,10,(1,))

tensor([1])

In [2]:
#Create A Train-Val set
import os
from sklearn.model_selection import train_test_split
imgs = os.listdir('datasets/playing_cards_minimal/train')
X_train, X_val = train_test_split(imgs, test_size=0.2, random_state=42)

#Move the val images to the val folder
import shutil
for img in X_val:
    shutil.move('datasets/playing_cards_minimal/train/'+img, 'datasets/playing_cards_minimal/val/'+img)




In [3]:
len(X_train)

2964

In [4]:
len(X_val)

741